# Basic Geographic Information System (GIS) functionality in Python

## Libraries and settings

In [ ]:
# Libraries
import os
import folium
import openpyxl
import platform
import pandas as pd
import geopandas as gpd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import a map of municipalities

In [ ]:
# Polygonmap als .json-File
polys = gpd.read_file("GEN_A4_GEMEINDEN_2019_epsg4326.json")

# Structure and type
print("nrows, ncols", polys.shape)
print("-------------------------------------------------------")
print("Type:", type(polys))

# Object 'polys' is a GeoDataFrame
polys.head()

## Plotting the map

In [ ]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
folium.Choropleth(
    geo_data=polys,
    name='polys',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Creating a spatial subset

In [ ]:
# Subset is formed by using indexing
idx = polys[polys['NAME'] == 'Winterthur'].index[0]
polys.iloc[[idx]]

## Plotting the spatial subset

In [ ]:
m = folium.Map(location=[47.44, 8.65], zoom_start=11)

# Map settings
folium.Choropleth(
    geo_data=polys.iloc[[idx]],
    name='polys',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Importing municipality data

In [ ]:
data = pd.read_excel('municipalities_kt_zh_data.xlsx', index_col=None)
print(type(data))

data.head(5)

## Creating a choropleth map

In [ ]:
# Variable to plot
var = 'residents_per_km2'

# Function for hiding the default legend
def folium_del_legend(choropleth: folium.Choropleth):
    del_list = []
    for child in choropleth._children:
        if child.startswith('color_map'):
            del_list.append(child)
            for del_item in del_list:
                choropleth._children.pop(del_item)
                return choropleth

# Bins for the color range
bins = list(data[var].quantile([0.00, 0.25, 0.50, 0.75, 1.00]))

# Initialize map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map-Settings (key_on contains the key to match the attribute data)
folium.Choropleth(
        geo_data=polys,
        name='choropleth',
        data=data,
        columns=['BFS', var],
        key_on='feature.properties.BFS',
        fill_color='RdGy',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name=var,
        bins=bins,
        reset=True
).add_to(m)

# Layer controls
folium.LayerControl().add_to(m)

# Plotting the map
m

In [ ]:
# Create a map with the absolute number of inhabitants and color it (choropleth map). 
# Extract and display population figures
einwohner_zuerich = data.loc[data['municipality_name'] == 'Zürich', 'residents'].iloc[0]
einwohner_uster = data.loc[data['municipality_name'] == 'Uster', 'residents'].iloc[0]

print(f"Absolute Einwohnerzahl von Zürich: {einwohner_zuerich}")
print(f"Absolute Einwohnerzahl von Uster: {einwohner_uster}")

# Extract the data for Zurich and Uster
data_subset = data[data['municipality_name'].isin(['Zürich', 'Uster'])]

# Create a map centered around Zurich and Uster
m = folium.Map(location=[47.36667, 8.55], zoom_start=10)

# Define the bins for the choropleth map based on the population data
# You may need to adjust these bins based on your actual data
bins = list(data_subset['residents'].quantile([0, 0.25, 0.5, 0.75, 1]))

# Add the choropleth layer
folium.Choropleth(
    geo_data=polys,
    name='choropleth',
    data=data_subset,
    columns=['BFS', 'residents'],  # Assuming 'BFS' is the identifier in your data
    key_on='feature.properties.BFS',
    fill_color='YlOrRd',  # Using a string for a built-in color palette
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of inhabitants',
    bins=bins,
    reset=True
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')